In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import src.graph_ops as go
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.autograd import Variable
from torch.utils import data
from sklearn.preprocessing import OneHotEncoder

In [2]:
graphs = np.load("graphs_updated.npy")
# options: graph_10+20k, graph_10+10k
labels = np.load("labels.npy")
# options: labels_10+20k, labels_10+10k

In [3]:
X = graphs
X = X[:,None]

In [4]:
first = pd.get_dummies(labels[:,0])
first[18] = 0
first[19] = 0
y = (first + pd.get_dummies(labels[:,1])).values
first = None

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [6]:
def cross_entropy(input, target, size_average=True):
    return torch.mean(-target * torch.log(input))


class CNN(nn.Module): 
    
    #constructor
    #take in X as a parameter
    def __init__(self, X, y, weight_decay = 0.0005):
        super(CNN, self).__init__()
        
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        #self.device = 'cpu'
        
        y_dim = y.shape[-1]
        
        self.conv1 = torch.nn.Conv2d(1, 18, kernel_size=3, stride=1, padding=1)
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        
        self.fc1 = torch.nn.Linear(18 * 10 * 10, 64)
        
        self.fc2 = torch.nn.Linear(64, y_dim)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

        
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay = weight_decay)
        self.to(self.device)
        
    # 1. input X
    def forward(self, X):
        X = F.relu(self.conv1(X))
        
        X = self.pool(X)
    
        X = X.view(-1, int(18 * 10 * 10))
        
        X = self.relu(self.fc1(X))
        
        #Computes the second fully connected layer (activation applied later)
        #Size changes from (1, 64) to (1, 10)
        X = self.fc2(X)
        X = self.sigmoid(X)
        X = 2*X/(X.sum(1)[:,None])
        return X
    
    def loss(self, pred, true):
        #PyTorch's own cross entropy loss function.
        score = cross_entropy
        l = None
        for i in range(true.shape[1]):
            if l is None:
                l = score(pred[:,i], true[:,i])
            else:
                l += score(pred[:,i], true[:,i])
        return l
    

    def fit(self, X, y, early_stopping = True, patience = 50):
        
        if isinstance(X, pd.DataFrame):
            X_arr = X.values
            X_tens = Variable(torch.Tensor(X_arr).float())
        else:
            X_tens = Variable(torch.Tensor(X).float())
        if isinstance(y, pd.DataFrame) or isinstance(y, pd.Series):
            y_arr = y.values
            if len(y_arr.shape) == 1:
                y_arr = y_arr[:,None]
            y_tens = Variable(torch.Tensor(y_arr).float())
        else:
            if len(y.shape) == 1:
                y = y[:,None]
            y_tens = Variable(torch.Tensor(y).float())
        
        X_tens = X_tens.to(self.device)
        y_tens = y_tens.to(self.device)
        
        if early_stopping == True:
            
            inds = np.arange(len(X_tens))
            num_train = int(len(X_tens) * .9)
            train_inds = np.random.choice(inds, num_train, replace=False)
            val_inds = np.setdiff1d(inds, train_inds)
            X_train = X_tens[train_inds]
            y_train = y_tens[train_inds]
            X_val = X_tens[val_inds]
            y_val = y_tens[val_inds]
            
        for epoch in range(1000):
            self.optimizer.zero_grad()
            if early_stopping==True:
                out_train = self.forward(X_train)
                loss = self.loss(out_train, y_train)
            if early_stopping==False:
                out_train = self.forward(X_tens)
                loss = self.loss(out_train, y_tens)
                print(loss)
                loss.backward()
                self.optimizer.step()
                continue
                
            out_val = self.forward(X_val)
            loss_val = self.loss(out_val, y_val)
            print(loss_val)


            if epoch == 0:
                min_loss = loss_val
                best_weights = self.state_dict()
                counter = 0
            else:
                if loss_val < min_loss:
                    min_loss = loss_val
                    best_weights = self.state_dict()
                    counter = 0
                else:
                    counter += 1
                    if counter == patience:
                        self.load_state_dict(best_weights)
                        return
                
            loss.backward()
            self.optimizer.step()
    
    def predict_proba(self, X):
        if isinstance(X, pd.DataFrame):
            X_arr = X.values
            X_tens = Variable(torch.Tensor(X_arr).float())
        else:
            X_tens = Variable(torch.Tensor(X).float())
            
        X_tens = X_tens.to(self.device)
        
        predict_out = self.forward(X_tens)
        temp =  predict_out.cpu().detach().numpy()
        temp[temp > 1-1e-6] = 1-1e-6
        temp[temp < 1e-6]=1e-6
        return temp
    
    def predict(self, X):
        proba = self.predict_proba(X).argsort(1)[:,-2:]
        proba.sort(1)
        return proba
    
    def score(self, X, y):
        y_pred = self.predict(X)
        y_true = y.argsort(1)[:,-2:]
        y_true.sort(1)
        return (y_pred == y_true).all(1).mean().item()
    
    def top_k_acc(self, X,y, k = 5):
        """Get top k accuracy for prediction. I.e., were the 2 correct nodes 
        in the top k returned. Note, k=2 is the score function."""
        y_pred = self.predict_proba(X)
        y_true = y.argsort(1)[:,-2:]  
        y_true.sort(1)
        eqs_list = []
        top_k = y_pred.argsort(1)[:,-k:]
        for i in range(y_true.shape[0]):
            row = y_true[i]
            eqs_arr = []
            for ele in row:
                eqs_arr.append(np.any(ele == top_k[i]))
            eqs_arr = np.array(eqs_arr)
            eq = eqs_arr.all()
            eqs_list.append(eq)
        eqs = np.array(eqs_list)

        return eqs.mean()

In [7]:
cnn = CNN(X, y)

cnn.fit(X_train,y_train)

tensor(4.6818, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.6394, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.0451, device='cuda:0', grad_fn=<AddBackward0>)
tensor(3.1128, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7163, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.8057, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.7702, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.6730, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.6216, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.6103, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.6088, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5952, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5748, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5583, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5485, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5403, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5301, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.5170, device='cuda:0', grad_fn=<AddBack

tensor(1.6942, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7002, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7021, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.6978, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.6974, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7061, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7048, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7016, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7144, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7199, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7620, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.8343, device='cuda:0', grad_fn=<AddBackward0>)
tensor(2.2686, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.7990, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.8375, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.8728, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.9272, device='cuda:0', grad_fn=<AddBackward0>)
tensor(1.8315, device='cuda:0', grad_fn=<AddBack

In [8]:
cnn.score(X_test,y_test)

0.6992

In [9]:
cnn.top_k_acc(X_test, y_test, k = 5)

0.8224